In [2]:
import os
from util.postgres import getPlans
def load_sql(query_path):
    """
    :param file_list: list of query file in str
    :return: list of sql query string
    """
    sqls = []
    with open(query_path, 'r') as f:
        for line in f.readlines():
            arr = line.strip().split("#####")
            sqls.append((arr[0], arr[1]))
    print("Read", len(sqls), "test queries.")
    return sqls

sqls = load_sql('./train/training_query/job.txt')
for i,sql in enumerate(sqls):
    print(i)
    getPlans(sql[1], None,check_hint_used=False,ENABLE_LEON=True, curr_file=sql[0])

Read 1000 test queries.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


AssertionError: canceling statement due to user request


In [1]:
import pickle
with open('./log/exp_v2.pkl', 'rb') as f:
    exp = pickle.load(f)

In [2]:
sqls = []
print("Length of exp:", len(exp['cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t']))
for item in exp['cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t']:
    # print("*"*20)
    node, encoding, attn = item
    # print(node.info['latency'])
    # print(node.cost)
    # print(node.info['sql_str'])
    sqls.append(node.info['sql_str'])


q36 = list(set(sqls))[1]
print("length of sqls:", len([i for i in sqls if i == q36]))
for item in exp['cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t']:
    print("*"*20)
    if ("cn.country_code)::text <> '[de]'::text)" in node.info['sql_str']):
        print("Q36")
    node, encoding, attn = item
    print(node.info['latency'])
    print(node.cost)
    # print(node.info['sql_str'])
    

Length of exp: 93
length of sqls: 17
********************
3240.864
25147.540669
********************
4348.969
25852.404174
********************
4401.098
26991.798643
********************
3691.521
25148.532544
********************
4074.796
25956.899662
********************
3914.095
25159.229489
********************
3915.618
25808.932294
********************
3983.113
25162.961169
********************
4390.488
25844.696643
********************
3646.769
25163.910669
********************
7709.49
1050359.307327
********************
4353.71
25178.344669
********************
7627.494
1049324.408346
********************
4201.248
25185.462444
********************
7505.775
1049176.440978
********************
4120.389
25185.462444
********************
1297.377
27160.907165
********************
1000000
27738.136112
********************
1215.168
27113.582004
********************
1000000
30754.921771
********************
1255.355
27117.281379
********************
1000000
28055.086073
****************

In [3]:
list(set(sqls))

["SELECT cn.name,mi_idx.info,t.title   FROM   info_type AS it1, complete_cast AS cc, comp_cast_type AS cct1, comp_cast_type AS cct2, movie_info_idx AS mi_idx, info_type AS it2, title AS t, movie_info AS mi, movie_companies AS mc, company_name AS cn, company_type AS ct, movie_keyword AS mk, keyword AS k, kind_type AS kt  WHERE kt.id = t.kind_id AND t.id = mi.movie_id AND t.id = mk.movie_id AND t.id = mi_idx.movie_id AND t.id = mc.movie_id AND t.id = cc.movie_id AND mk.movie_id = mi.movie_id AND mk.movie_id = mi_idx.movie_id AND mk.movie_id = mc.movie_id AND mk.movie_id = cc.movie_id AND mi.movie_id = mi_idx.movie_id AND mi.movie_id = mc.movie_id AND mi.movie_id = cc.movie_id AND mc.movie_id = mi_idx.movie_id AND mc.movie_id = cc.movie_id AND mi_idx.movie_id = cc.movie_id AND k.id = mk.keyword_id AND it1.id = mi.info_type_id AND it2.id = mi_idx.info_type_id AND ct.id = mc.company_type_id AND cn.id = mc.company_id AND cct1.id = cc.subject_id AND cct2.id = cc.status_id AND ((it1.info)::tex

In [2]:
"""
Tree Transformer Block
"""
import util.postgres as postgres
from util import envs
from util import plans_lib
import os
import pickle

# load workload and queryFeaturizer
if not os.path.exists('./log/workload_job_training.pkl'):
    workload = envs.JoinOrderBenchmark_Train(envs.JoinOrderBenchmark_Train.Params())
    workload.workload_info.table_num_rows = postgres.GetAllTableNumRows(workload.workload_info.rel_names)
    workload.workload_info.alias_to_names = postgres.GetAllAliasToNames(workload.workload_info.rel_ids)
    print(workload.workload_info)
    # dump queryFeaturizer and workload
    with open('./log/workload_job_training.pkl', 'wb') as f:
        pickle.dump(workload, f)
else:
    
    with open('./log/workload_job_training.pkl', 'rb') as f:
        workload = pickle.load(f)
queryFeaturizer = plans_lib.QueryFeaturizer(workload.workload_info)

for item in exp['cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t']:
    # print("*"*20)
    node, encoding, attn = item
    plans_lib.GatherUnaryFiltersInfo(node)
    postgres.EstimateFilterRows(node)  
    print(queryFeaturizer(node).shape)
    break


/data1/chenxu/miniconda3/envs/leon/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-26 20:36:29,274	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


PostgreSQL config:
(666,)


In [3]:
from util.postgres import getPlans
from util.dataset import *
from util.model import *
from leon_pl import load_callbacks
from test_case import *
import pytorch_lightning.loggers as pl_loggers
import os
from leon_experience import TIME_OUT

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
Transformer_model = SeqFormer(
                        input_dim=configs['node_length'],
                        hidden_dim=256,
                        output_dim=1,
                        mlp_activation="ReLU",
                        transformer_activation="gelu",
                        mlp_dropout=0.1,
                        transformer_dropout=0.1,
                        query_dim=666,
                        node_embedding_dim=30,
                        padding_size=40
                    )

def Getpair(exp, key=None):
    pairs = []
    if key:
        for j in exp[key]:
            for k in exp[key]:
                ############################NEW
                if j[0].info['sql_str'] != k[0].info['sql_str']:
                    continue    

                if (j[0].info['sql_str'] == k[0].info['sql_str']) and (j[0].hint_str() == k[0].hint_str()): # sql 和 hint 都相同
                    continue

                if j[0].info['sql_str'] != k[0].info['sql_str'] and (j[0].info['latency'] == TIME_OUT or k[0].info['latency'] == TIME_OUT):
                        continue
                # if (j[0].info['latency'] == k[0].info['latency']): # latency 相同 1s之内不把他train_pair
                if max(j[0].info['latency'],k[0].info['latency']) / min(j[0].info['latency'],k[0].info['latency']) < 1.2:
                    continue
                # if j[0].info['latency'] == 90000 or k[0].info['latency'] == 90000:
                #     continue
                tem = [j, k]
                pairs.append(tem)
    else:          
        for eq in exp.keys():
            for j in exp[eq]:
                for k in exp[eq]:
                    if j[0].info['sql_str'] != k[0].info['sql_str']:
                        continue    

                    if (j[0].info['sql_str'] == k[0].info['sql_str']) and (j[0].hint_str() == k[0].hint_str()): # sql 和 hint 都相同
                        continue
                    if j[0].info['sql_str'] != k[0].info['sql_str'] and (j[0].info['latency'] == TIME_OUT or k[0].info['latency'] == TIME_OUT):
                        continue
                    # if (j[0].info['latency'] == k[0].info['latency']): # latency 相同 1s之内不把他train_pair
                    if max(j[0].info['latency'],k[0].info['latency']) / min(j[0].info['latency'],k[0].info['latency']) < 1.2:
                        continue
                    # if j[0].info['latency'] == 90000 or k[0].info['latency'] == 90000:
                    #     continue
                    tem = [j, k]
                    pairs.append(tem)
    return pairs

prev_optimizer_state_dict = None
model = Transformer_model
model = PL_Leon(model, prev_optimizer_state_dict)

/data1/chenxu/miniconda3/envs/leon/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:

# key = 'cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t'

# new_exp = {key:[]}
# for item in exp['cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t']:
#     node, encoding, attn = item
#     if ("cn.country_code)::text <> '[de]'::text)" in node.info['sql_str']):
#         # print("Q36")
#         continue
#     new_exp[key].append(item)

# print(len(new_exp[key]))

logger = pl_loggers.WandbLogger(save_dir=os.getcwd() + '/logs', name="test", project='leon3')

key = None
train_pairs = Getpair(exp, key=key)
leon_dataset = prepare_dataset(train_pairs, queryFeaturizer=queryFeaturizer)
dataloader_train = DataLoader(leon_dataset, batch_size=512, shuffle=True, num_workers=0)
# dataloader_val = DataLoader(leon_dataset, batch_size=512, shuffle=False, num_workers=0)
dataset_val = BucketDataset(exp, keys=key)
batch_sampler = BucketBatchSampler(dataset_val.buckets, batch_size=1)
dataloader_val = DataLoader(dataset_val, batch_sampler=batch_sampler)


In [5]:
trainer = pl.Trainer(accelerator="gpu",
                        devices=[0],
                        max_epochs=500,
                        callbacks=None,
                        logger=logger)

# trainer.fit(model, dataloader_train, dataloader_val)
trainer.fit(model, dataloader_train)
prev_optimizer_state_dict = trainer.optimizers[0].state_dict()

print("*"*20)
print("Current Accuracy For Each EqSet: ", model.eq_summary)
print("*"*20)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


/data1/chenxu/miniconda3/envs/leon/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type      | Params
------------------------------------
0 | model | SeqFormer | 196 K 
------------------------------------
196 K     Trainable params
0         Non-trainable params
196 K     Total params
0.785     Total estimated model params size (MB)
/data1/chenxu/miniconda3/envs/leon/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/data1/chenxu/miniconda3/envs/leon/lib/python3.8/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 499: 100%|██████████| 20/20 [00:00<00:00, 26.74it/s, v_num=j0s1]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 20/20 [00:00<00:00, 26.01it/s, v_num=j0s1]
********************
Current Accuracy For Each EqSet:  {}
********************


3个sql
********************
Current Accuracy For Each EqSet:  {'cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t': (0.8683127572016461, 243)}
********************

2个sql 不要q36
********************
Current Accuracy For Each EqSet:  {'cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t': (0.7298444130127298, 707)}
********************

3个sql overfit train 500 epoch
********************
Current Accuracy For Each EqSet:  {'cc,cct1,cct2,cn,ct,it1,it2,k,kt,mc,mi,mi_idx,mk,t': (0.7086280056577087, 707)}
********************